In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect(
    database = 'postgres_db',
    user = 'postgres_user',
    password = 'admin',
    host = 'client.tuple-mia.com',
    port = '5432',
    sslmode = 'require')

In [3]:
temp = conn.cursor()

In [4]:
import yaml
with open('/home/ubuntu/MappingBuffer_event.yml','r') as f:
    doc = yaml.load(f)

In [5]:
print doc['table_map']
print doc['column_map']

{'CUSTOMER_CONTACT': 'contact', 'EVENT_LOG': 'event', 'TRANSACTION_MASTER': 'trans', 'CUSTOMER_SECONDARY': 'temp_customer_secondary', 'EVENT_MASTER': 'event_master', 'CUSTOMER_MASTER': 'users'}
{'CUSTOMER_CONTACT': {'phone_number': 'phone', 'cust_id': 'id', 'firstname': 'first_name', 'facebook_id': 'facebook_id', 'email_id': 'email', 'lastname': 'last_name'}, 'EVENT_LOG': {'timestamp': 'timestamp', 'cust_id': 'from_id', 'product_id': 'to_id', 'action_type': 'actiontype'}, 'TRANSACTION_MASTER': {'cust_id': 'userid', 'product_id': 'product_id', 'revenue': 'revenue', 'timestamp': 'purchase_date', 'renewal': 'renewal', 'quantity': 'quantity'}, 'CUSTOMER_SECONDARY': {'cust_id': 'cust_id'}, 'EVENT_MASTER': {'action_category_id': None, 'action_category_name': None, 'action_type': 'actiontype', 'action_name': 'action_desc'}, 'CUSTOMER_MASTER': {'create_date': 'create_date', 'birthday': 'birthday', 'cust_id': 'id'}}


In [6]:
query = 'SELECT * FROM %s'%(doc['table_map']['TRANSACTION_MASTER'])

In [7]:
print query

SELECT * FROM trans


In [8]:
temp.execute(query)

In [9]:
trans = temp.fetchall()

In [10]:
len(trans)

4428

In [11]:
col_names = [desc[0] for desc in temp.description]

In [12]:
col_names

['product_id',
 'receipt_id',
 'revenue',
 'purchase_date',
 'renewal',
 'userid',
 'insider',
 'platform',
 'quantity']

In [13]:
import pandas as pd

In [14]:
trans_table = pd.DataFrame(trans, columns=col_names)

In [15]:
trans_table.head()

,product_id,receipt_id,revenue,purchase_date,renewal,userid,insider,platform,quantity
0,com.gopaktor.subscription.v1.premium.1m,1984955,31.28,2016-09-07 13:59:45+00:00,0,3893506,0,ios,0
1,com.gopaktor.subscription.v1.premium.1m,2170195,31.28,2016-09-25 03:06:11+00:00,0,3793254,0,ios,0
2,com.gopaktor.subscription.v1.premium.1m,2259995,31.28,2016-10-01 02:27:37+00:00,0,6148794,0,ios,0
3,com.gopaktor.subscription.v4.premium.1m,1980761,31.28,2016-10-01 06:39:16+00:00,0,6149220,0,android,0
4,com.gopaktor.subscription.v4.premium.1m,1980865,31.28,2016-10-01 09:07:49+00:00,0,6149432,0,android,0


In [16]:
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['cust_id']:'cust_id'}, inplace=True)
trans_table.rename(columns={doc['column_map']['TRANSACTION_MASTER']['product_id']:'product_id'}, inplace=True)

In [17]:
trans_table.head()

,product_id,receipt_id,revenue,purchase_date,renewal,cust_id,insider,platform,quantity
0,com.gopaktor.subscription.v1.premium.1m,1984955,31.28,2016-09-07 13:59:45+00:00,0,3893506,0,ios,0
1,com.gopaktor.subscription.v1.premium.1m,2170195,31.28,2016-09-25 03:06:11+00:00,0,3793254,0,ios,0
2,com.gopaktor.subscription.v1.premium.1m,2259995,31.28,2016-10-01 02:27:37+00:00,0,6148794,0,ios,0
3,com.gopaktor.subscription.v4.premium.1m,1980761,31.28,2016-10-01 06:39:16+00:00,0,6149220,0,android,0
4,com.gopaktor.subscription.v4.premium.1m,1980865,31.28,2016-10-01 09:07:49+00:00,0,6149432,0,android,0


In [18]:
trans_df = trans_table[['cust_id', 'product_id']]

In [19]:
trans_df.head()

,cust_id,product_id
0,3893506,com.gopaktor.subscription.v1.premium.1m
1,3793254,com.gopaktor.subscription.v1.premium.1m
2,6148794,com.gopaktor.subscription.v1.premium.1m
3,6149220,com.gopaktor.subscription.v4.premium.1m
4,6149432,com.gopaktor.subscription.v4.premium.1m


In [20]:
len(trans_df.cust_id.unique())

3087

In [21]:
len(trans_df.product_id.unique())

18

In [43]:
grp_prod = trans_df.groupby(['product_id'])

In [44]:
grp_cust = trans_df.groupby(['cust_id'])

In [45]:
grp_cust_df = grp_cust.count()

In [46]:
grp_prod_df = grp_prod.count()

In [47]:
grp_prod_df = grp_prod_df.reset_index()

In [48]:
grp_prod_df

,product_id,cust_id
0,com.gopaktor.credit.v2.1000,185
1,com.gopaktor.credit.v2.10000,3
2,com.gopaktor.credit.v2.3000,6
3,com.gopaktor.credit.v2.6000,1
4,com.gopaktor.credit.v3.2000,1
5,com.gopaktor.subscription.v1.premium.1m,1536
6,com.gopaktor.subscription.v11.premium.6m,43
7,com.gopaktor.subscription.v12.premium.1m,341
8,com.gopaktor.subscription.v12.premium.3m,132
9,com.gopaktor.subscription.v2.premium.1m.2000,696


In [49]:
grp_prod_df.rename(columns={'cust_id':'count'}, inplace=True)

In [50]:
grp_cust_df = grp_cust_df.reset_index()

In [51]:
grp_cust_df.rename(columns = {'product_id':'count'}, inplace=True)

In [52]:
grp_cust_df.head(20)

,cust_id,count
0,2942914,1
1,3053432,10
2,3129800,1
3,3289765,1
4,3325662,2
5,3409109,2
6,3793254,4
7,3893506,4
8,4478184,2
9,6148771,1


In [55]:
grp_prod_df['perc'] = grp_prod_df['count']/grp_prod_df['count'].sum()*100

In [57]:
grp_prod_df

,product_id,count,perc
0,com.gopaktor.credit.v2.1000,185,4.177958
1,com.gopaktor.credit.v2.10000,3,0.067751
2,com.gopaktor.credit.v2.3000,6,0.135501
3,com.gopaktor.credit.v2.6000,1,0.022584
4,com.gopaktor.credit.v3.2000,1,0.022584
5,com.gopaktor.subscription.v1.premium.1m,1536,34.688347
6,com.gopaktor.subscription.v11.premium.6m,43,0.971093
7,com.gopaktor.subscription.v12.premium.1m,341,7.700994
8,com.gopaktor.subscription.v12.premium.3m,132,2.981030
9,com.gopaktor.subscription.v2.premium.1m.2000,696,15.718157


In [67]:
grp_cust_df['perc'] = grp_cust_df['count']/grp_cust_df['count'].sum()*100

In [68]:
grp_cust_df.head()

,cust_id,count,perc
0,2942914,1,0.022584
1,3053432,10,0.225836
2,3129800,1,0.022584
3,3289765,1,0.022584
4,3325662,2,0.045167


In [69]:
sum(grp_prod_df.perc)

100.0

In [70]:
import numpy as np

In [94]:
grp_prod_df['temp'] = ""

In [97]:
grp_prod_df.loc[grp_prod_df['count'] > grp_prod_df['count'].quantile(0.9), 'temp'] = 1

In [98]:
grp_prod_df

,product_id,count,perc,temp
0,com.gopaktor.credit.v2.1000,185,4.177958,
1,com.gopaktor.credit.v2.10000,3,0.067751,
2,com.gopaktor.credit.v2.3000,6,0.135501,
3,com.gopaktor.credit.v2.6000,1,0.022584,
4,com.gopaktor.credit.v3.2000,1,0.022584,
5,com.gopaktor.subscription.v1.premium.1m,1536,34.688347,1
6,com.gopaktor.subscription.v11.premium.6m,43,0.971093,
7,com.gopaktor.subscription.v12.premium.1m,341,7.700994,
8,com.gopaktor.subscription.v12.premium.3m,132,2.981030,
9,com.gopaktor.subscription.v2.premium.1m.2000,696,15.718157,


In [77]:
grp_cust_df['temp'] = ""

In [99]:
grp_cust_df.loc[grp_cust_df['count'] > grp_prod_df['count'].quantile(0.2), 'temp'] = 1

In [105]:
list(grp_prod_df[grp_prod_df['count'] > grp_prod_df['count'].quantile(0.2)].product_id)

['com.gopaktor.credit.v2.1000',
 'com.gopaktor.credit.v2.3000',
 'com.gopaktor.subscription.v1.premium.1m',
 'com.gopaktor.subscription.v11.premium.6m',
 'com.gopaktor.subscription.v12.premium.1m',
 'com.gopaktor.subscription.v12.premium.3m',
 'com.gopaktor.subscription.v2.premium.1m.2000',
 'com.gopaktor.subscription.v2.premium.3m',
 'com.gopaktor.subscription.v2.premium.3m.2000',
 'com.gopaktor.subscription.v2.premium.6m',
 'com.gopaktor.subscription.v2.premium.6m.2000',
 'com.gopaktor.subscription.v3.premium.3m',
 'com.gopaktor.subscription.v3.premium.6m',
 'com.gopaktor.subscription.v4.premium.1m']

In [108]:
list(grp_cust_df[grp_cust_df['count'] > grp_cust_df['count'].quantile(0.1)].cust_id)

['3053432',
 '3325662',
 '3409109',
 '3793254',
 '3893506',
 '4478184',
 '6149073',
 '6149179',
 '6149287',
 '6149306',
 '6149386',
 '6149464',
 '6149487',
 '6149539',
 '6150290',
 '6150561',
 '6150777',
 '6150881',
 '6150928',
 '6150990',
 '6151290',
 '6151715',
 '6151910',
 '6151944',
 '6152091',
 '6152401',
 '6152619',
 '6152623',
 '6152634',
 '6152796',
 '6153411',
 '6153466',
 '6153473',
 '6153514',
 '6153549',
 '6154007',
 '6154152',
 '6154790',
 '6155049',
 '6155068',
 '6155107',
 '6155110',
 '6155111',
 '6155578',
 '6155723',
 '6155741',
 '6155796',
 '6155837',
 '6156020',
 '6156187',
 '6156524',
 '6156840',
 '6156944',
 '6157463',
 '6157606',
 '6157771',
 '6157948',
 '6158059',
 '6158155',
 '6158169',
 '6158339',
 '6158490',
 '6158687',
 '6158890',
 '6158975',
 '6159290',
 '6159474',
 '6159515',
 '6160008',
 '6160293',
 '6160644',
 '6160965',
 '6161492',
 '6162055',
 '6162348',
 '6162458',
 '6163002',
 '6163171',
 '6163185',
 '6163632',
 '6163893',
 '6163905',
 '6163941',
 '61